### Pinecone Vector DB

In [ ]:
api_key="PineconeAPIKey"

In [ ]:
!pip install -qU langchain langchain-pinecone langchain-openai

In [ ]:
from pinecone import Pinecone

pc=Pinecone(api_key=api_key)

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024, api_key="OpenAIAPIKey")

In [ ]:
from pinecone import ServerlessSpec

index_name = "rag"

if not pc.has_index(index_name):
  pc.create_index(
      name=index_name,
      dimension=1024,
      metric="cosine",
      spec=ServerlessSpec(cloud="aws", region="us-east-1")
  )

  index = pc.Index(index_name)

In [ ]:
index = pc.Index(index_name)
index

In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
vector_store

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [ ]:
vector_store.add_documents(documents=documents)

['496dda62-954e-462e-9422-40399b15a8cd',
 '36a47946-61b3-4d9b-8b46-78ee747de253',
 '649ab5ad-e4d1-4d14-ac93-bb5f2102a80c',
 'd85c3598-9726-40bf-879b-b516cba93ac8',
 '3c02d8b0-5944-4138-8665-af68eef1eea1',
 'f7ee9e53-75d3-4663-8592-878c1e476d4b',
 '3cbee80b-1939-49c6-b9bc-a987cac946bb',
 '874d445a-77c6-46d9-8565-1fcb175e66f1',
 '40d78e3c-9908-40a1-b2ca-9ed3b3f0b518',
 '737cff4d-48f3-49c1-acc5-6b503a2e8e7c']

In [ ]:
### Query Directly

results = vector_store.similarity_search(
    "Langchain provides agstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"}
)

for res in results:
  print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
results = vector_store.similarity_search_with_score(
     "Langchain provides agstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"}
)
for res,score in results:
  print(f"*[SIM={score:3f}] {res.page_content} [{res.metadata}]")

*[SIM=0.468836] LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
*[SIM=0.396116] Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [ ]:
### Retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2, "score_threshold": 0.4}
)
retriever.invoke("Stealing from the band is a crime", filter={"source": "news"})

[Document(id='d85c3598-9726-40bf-879b-b516cba93ac8', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(id='40d78e3c-9908-40a1-b2ca-9ed3b3f0b518', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.')]

# Astra db is in colab